In [5]:
import pandas as pd
import numpy as np
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import os


data = pd.read_csv("ner_dataset.csv", encoding="latin1")


In [2]:
data = data.fillna(method="ffill")

In [3]:
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [8]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
words.sort()
n_words = len(words); n_words

35179

In [9]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
        
        
# class NERModel(object):
#     def __init__(self,data):
#         self.sentenceGetter = SentenceGetter(data)
#         self.model = modelSpec()
        
#     def modelSpec():
#         input = Input(shape=(max_len,))
#         model = Embedding(input_dim=n_words + 1, output_dim=20,input_length=max_len, mask_zero=True)(input)
#         model = Bidirectional(LSTM(units=50, return_sequences=True,recurrent_dropout=0.1))(model)
#         model = TimeDistributed(Dense(50, activation="relu"))(model)  
#         crf = CRF(n_tags)
#         out = crf(model)
#         model = Model(input, out)
#         model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
#         return model
        
        

In [10]:
tags = list(set(data["Tag"].values))
tags.sort()


In [11]:
n_tags = len(tags); n_tags

17

In [12]:
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)


[('Thousands', 'NNS', 'O')]


In [13]:
sentences = getter.sentences

In [14]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}


In [12]:
word2idx["Obama"]

11522

In [13]:
tag2idx["B-geo"]

2

In [14]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [15]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [16]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [17]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [18]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [19]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [149]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=50,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [15]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer


In [16]:
model = Model(input, out)

In [17]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 75, 50)            1758950   
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 75, 100)           60400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 75, 17)            1717      
Total params: 1,821,067
Trainable params: 1,821,067
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=20, validation_split=0.1, verbose=1)

Train on 38846 samples, validate on 4317 samples
Epoch 1/20
38846/38846 [==============================] - 218s 6ms/step - loss: 0.1180 - acc: 0.9736 - val_loss: 0.0523 - val_acc: 0.9849
Epoch 2/20
38846/38846 [==============================] - 119s 3ms/step - loss: 0.0425 - acc: 0.9876 - val_loss: 0.0424 - val_acc: 0.9873
Epoch 3/20
38846/38846 [==============================] - 196s 5ms/step - loss: 0.0365 - acc: 0.9893 - val_loss: 0.0393 - val_acc: 0.9881
Epoch 4/20
38846/38846 [==============================] - 144s 4ms/step - loss: 0.0337 - acc: 0.9900 - val_loss: 0.0383 - val_acc: 0.9884
Epoch 5/20
38846/38846 [==============================] - 120s 3ms/step - loss: 0.0320 - acc: 0.9905 - val_loss: 0.0384 - val_acc: 0.9881
Epoch 6/20
38846/38846 [==============================] - 120s 3ms/step - loss: 0.0305 - acc: 0.9909 - val_loss: 0.0375 - val_acc: 0.9885
Epoch 7/20
38846/38846 [==============================] - 94s 2ms/step - loss: 0.0295 - acc: 0.9912 - val_loss: 0.0373 - va

In [25]:
model.save('LSTM_NER.h5')

In [26]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
test_pred = model.predict(X_te, verbose=1)

idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

4796/4796 [==============================] - 3s 567us/step


In [27]:
print(classification_report(test_labels, pred_labels))

             precision    recall  f1-score   support

        gpe       0.95      0.94      0.94      1477
        geo       0.79      0.89      0.84      3750
        org       0.49      0.59      0.54      1963
        per       0.70      0.77      0.73      1716
        tim       0.84      0.77      0.81      2033
        art       0.10      0.12      0.11        43
        eve       0.33      0.25      0.29        24
        nat       0.86      0.40      0.55        15

avg / total       0.75      0.80      0.77     11021



In [97]:
for i in range(len(X_te)):
    p = model.predict(np.array([X_te[i]]))
    p = np.argmax(p, axis=-1)
    true = np.argmax(y_te[i], -1)
    fileName = "testFile"+str(i)
    file = open(fileName,'w')
    
    file.write("{:15}||{:5}||{}".format("Word", "True", "Pred"))
    file.write(30 * "=")
    for w, t, pred in zip(X_te[i], true, p[0]):
        if w != 0:
            file.write("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))
    file.close()